In [1]:
#### RUN THIS CODE IF YOU USE GOOGLE COLAB ####

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the material for this assignment,
# e.g. 'cvf20/assignments/assignment3/'
FOLDERNAME = 'NTS'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Make sure that the python modules in the assignment folder are found by the notebook:
import sys
import os
path_drive = os.path.join("/content/drive/My Drive", FOLDERNAME)
sys.path.append(path_drive)

# Copy the yeast-cells data in the content folder of the notebook:
dataset_path = os.path.join(path_drive, "data/CNN_DM") 
#dataset_path = dataset_path.replace(" ", "\ ")
#!cp -r $dataset_path ./

# Move to the main content folder:
#%cd /content

# Install tensorboardX:
!pip install tensorboardX

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 204kB 2.8MB/s 


In [0]:
import os
os.chdir('/content/drive/My Drive/NTS/codes')

In [0]:
import numpy as np
from collections import Counter
import numpy as np
import math
import torch
import torch.nn as nn
import random
import time
import io
import codecs

import rougee
from extractor import PacSumExtractorWithBert, PacSumExtractorWithTfIdf
from data_iterator import Dataset
import rougee

In [4]:
tune_dataset = Dataset('/content/drive/My Drive/NTS/data/CNN_DM/cd.validation.h5df', '/content/drive/My Drive/NTS/pacssum_models/vocab.txt')
tune_dataset_iterator = tune_dataset.iterate_once_doc_bert()

input max len : 60


In [0]:
tune_dataset_iterator = tune_dataset.iterate_once_doc_bert()

In [6]:
extractor = PacSumExtractorWithBert('/content/drive/My Drive/NTS/pacssum_models/pytorch_model_unfinetuned.bin', '/content/drive/My Drive/NTS/pacssum_models/bert_config.json')

odict_keys(['embeddings.word_embeddings.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.LayerNorm.gamma', 'embeddings.LayerNorm.beta', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.LayerNorm.gamma', 'encoder.layer.0.attention.output.LayerNorm.beta', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.LayerNorm.gamma', 'encoder.layer.0.output.LayerNorm.beta', 'encoder.layer.1.attention.self.query.weight', 'encoder.layer.1.attention.self.query.bias'

In [0]:
def _select_tops(edge_scores, beta, lambda1, lambda2):

        min_score = edge_scores.min()
        max_score = edge_scores.max()
        edge_threshold = min_score + beta * (max_score - min_score)
        new_edge_scores = edge_scores - edge_threshold
        forward_scores, backward_scores, _ = _compute_scores(new_edge_scores, 0)
        forward_scores = 0 - forward_scores

        paired_scores = []
        for node in range(len(forward_scores)):
            paired_scores.append([node,  lambda1 * forward_scores[node] + lambda2 * backward_scores[node]])

        #shuffle to avoid any possible bias
        random.shuffle(paired_scores)
        paired_scores.sort(key = lambda x: x[1], reverse = True)
        extracted = [item[0] for item in paired_scores[:3]]


        return extracted

def _compute_scores(similarity_matrix, edge_threshold):

    forward_scores = [0 for i in range(len(similarity_matrix))]
    backward_scores = [0 for i in range(len(similarity_matrix))]
    edges = []
    for i in range(len(similarity_matrix)):
        for j in range(i+1, len(similarity_matrix[i])):
            edge_score = similarity_matrix[i][j]
            if edge_score > edge_threshold:
                forward_scores[j] += edge_score
                backward_scores[i] += edge_score
                edges.append((i,j,edge_score))

    return np.asarray(forward_scores), np.asarray(backward_scores), edges

In [0]:
def _tune_extractor(edge_scores):

        tops_list = []
        hparam_list = []
        num = 10
        for k in range(num + 1):
            beta = k / num
            for i in range(11):
                lambda1 = i/10
                lambda2 = 1 - lambda1
                extracted = _select_tops(edge_scores, beta=beta, lambda1=lambda1, lambda2=lambda2)

                tops_list.append(extracted)
                hparam_list.append((beta, lambda1, lambda2))

        return tops_list, hparam_list

In [0]:
rouge = rougee.Rouge()

In [20]:
tune_dataset_iterator

<generator object Dataset.iterate_once_doc_bert at 0x7f2a34633ca8>

In [21]:
for item in tune_dataset_iterator:
    
    article, abstract, inputs = item

Processing file: /content/drive/My Drive/NTS/data/CNN_DM/cd.validation.h5df


In [9]:
example_num=1000


summaries, references = [], []
k = 0
for item1 in tune_dataset_iterator:
    
    article, abstract, inputs = item1
    edge_scores =extractor._calculate_similarity_matrix(*inputs)
    tops_list, hparam_list = _tune_extractor(edge_scores)

    summary_list = [list(map(lambda x: article[x], ids)) for ids in tops_list]
    summaries.append(summary_list)
    references.append([abstract])
    k += 1
    print(k)
    if k % example_num == 0:
        break

Processing file: /content/drive/My Drive/NTS/data/CNN_DM/cd.validation.h5df
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258


In [24]:
summaries

[]

In [12]:
count = 0
best_rouge = 0
best_hparam = None
for i in range(len(summaries[0])):
    print(i)
    print("threshold :  "+str(hparam_list[i])+'\n')
    #print("non-lead ratio : "+str(ratios[i])+'\n')
    for k in range(len(summaries)):
        #print(i)
        print(k)
        summ = summaries[k][i]
        #print('summ')
        #print(len(summ))
        #print(summaries)
        ref = references[k][0]
        
        if len(summ) == len(ref):
            count = count + 1
            
            print('sum')
            print(len(summ))
            if i != 8:
                if summaries[k][i][0]!='.' and summaries[k][i][1]!='.' and summaries[k][i][2]!='.':
                    result = rouge.get_scores(summ, ref, avg = True)
                    #print(result)
                    if result['rouge-l']['f'] > best_rouge:
                        best_rouge = result['rouge-l']['f']
                        best_hparam = hparam_list[i]

print("The best hyper-parameter :  beta %.4f , lambda1 %.4f, lambda2 %.4f " % (best_hparam[0], best_hparam[1], best_hparam[2]))
print("The best rouge_1_f_score :  %.4f " % best_rouge)

beta = best_hparam[0]
lambda1 = best_hparam[1]
lambda2 = best_hparam[2]

Streaming output truncated to the last 5000 lines.
868
869
sum
3
870
871
872
sum
3
873
sum
3
874
875
sum
3
876
877
878
sum
3
879
sum
3
880
881
sum
3
882
sum
3
883
sum
3
884
885
886
887
888
889
sum
3
890
891
892
893
894
895
sum
3
896
897
898
sum
3
899
900
901
902
903
904
905
906
907
908
909
sum
3
910
911
912
sum
3
913
914
915
916
917
sum
3
918
919
920
921
922
923
924
925
926
927
928
929
930
sum
3
931
932
933
sum
3
934
935
936
937
938
939
sum
3
940
941
sum
3
942
943
sum
3
944
945
946
sum
3
947
948
949
950
sum
3
951
952
sum
3
953
sum
3
954
955
956
957
sum
3
958
959
sum
3
960
961
962
963
964
sum
3
965
sum
3
966
967
968
sum
3
969
970
971
972
973
974
975
sum
3
976
sum
3
977
978
979
sum
3
980
981
982
sum
3
983
984
985
sum
3
986
sum
3
987
sum
3
988
989
990
991
992
sum
3
993
994
995
sum
3
996
sum
3
997
sum
3
998
999
118
threshold :  (1.0, 0.8, 0.19999999999999996)

0
sum
3
1
2
3
4
5
6
7
8
9
10
11
12
sum
3
13
14
15
16
17
18
19
sum
3
20
sum
3
21
22
sum
3
23
24
sum
3
25
sum
3
26
27
28
29
30
31
32


In [17]:
#testing
test_dataset = Dataset('/content/drive/My Drive/NTS/data/CNN_DM/cd.test.h5df', '/content/drive/My Drive/NTS/pacssum_models/vocab.txt')
test_dataset_iterator = test_dataset.iterate_once_doc_bert()

input max len : 60


In [0]:
test_dataset_iterator = test_dataset.iterate_once_doc_bert()

In [19]:
#Extract summary
from collections import Counter
import numpy as np
import math
import torch
import torch.nn as nn
import random
import time
import io
import codecs

extract_num = 3
summaries_test = []
references_test = []

for item in test_dataset_iterator:
    article, abstract, inputs = item
    if len(article) <= extract_num and len(article) == len(abstract):
        summaries.append(article)
        references.append([abstract])
        continue

    edge_scores = extractor._calculate_similarity_matrix(*inputs)
    ids = _select_tops(edge_scores = edge_scores, beta=0.9, lambda1=0.6, lambda2=0.4)
    summary = list(map(lambda x: article[x], ids))
    summaries_test.append(summary)
    references_test.append([abstract])

Processing file: /content/drive/My Drive/NTS/data/CNN_DM/cd.test.h5df


KeyboardInterrupt: ignored

In [20]:
len(summaries_test)

6927

In [25]:
new_score = 0
score = 0
for i in range (0,6000):
    print(i)
    for k in range (len(summaries_test[0])):
        if len(summaries_test[i]) == len(summaries_test[0]) and summaries_test[i][k]!= '.':
            new_score = rouge.get_scores(summaries_test[i][k], references_test[i][0][0])
            R_L_f = new_score[0]['rouge-l']['f']
            score = (score + R_L_f)

Streaming output truncated to the last 5000 lines.
hyp:
['ms hall is now urging others to check their inhalers before use']
ref
['alison hall , 48 , says the nail became lodged and she started to choke']
hyp:
['there is a millimetre gap on each side']
ref
['alison hall , 48 , says the nail became lodged and she started to choke']
hyp:
['there is a millimetre gap on each side']
ref
['alison hall , 48 , says the nail became lodged and she started to choke']
hyp:
['there is a millimetre gap on each side']
ref
['alison hall , 48 , says the nail became lodged and she started to choke']
5865
hyp:
['and nearly three fourths of this population is comprised of nonviolent offenders']
ref
['cory booker : the unfortunate reality is that the united states leads the world in incarceration , not education']
hyp:
['and nearly three fourths of this population is comprised of nonviolent offenders']
ref
['cory booker : the unfortunate reality is that the united states leads the world in incarceration , n

In [26]:
score/6000

0.40645415901360565